In [2]:
!pip install google-cloud-aiplatform google-cloud-storage pandas sklearn


In [19]:
pip install google-cloud-secret-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 1.3 MB/s eta 0:00:00


In [ ]:
import os

PROJECT_ID = 'molten-unison-414815'
BUCKET_NAME = 'renzo_bucket_test'
REGION = 'us'
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID


In [3]:
from google.cloud import storage
import pandas as pd

In [4]:
def descargar_datos(bucket_name, file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.download_to_filename(file_name)
    return pd.read_csv(file_name)


file_name = 'iris.csv'
df = descargar_datos(BUCKET_NAME, file_name)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [7]:

# Preprocesar los datos
le = LabelEncoder()
df['species'] = le.fit_transform(df['species'])

# Dividir los datos
X = df.drop('species', axis=1)
y = df['species']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
# Entrenar un modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [10]:
predicciones = model.predict(X_test)

In [13]:
from google.cloud import bigquery

client = bigquery.Client()
table_id = 'molten-unison-414815.iris_dataset.iris_table'


rows_to_insert = [
    {u"sepal_length": float(row[0]), u"sepal_width": float(row[1]), u"petal_length": float(row[2]), u"petal_width": float(row[3]), u"species": int(pred)} for row, pred in zip(X_test.to_numpy(), predicciones)
]

errors = client.insert_rows_json(table_id, rows_to_insert)
if errors == []:
    print("New rows have been added.")
else:
    print("Errors occurred:", errors)


New rows have been added.
